In [1]:
import time
START = time.time()

# **Now we will Create a final pipeline which will contain all the preprocessing and the model application**

In [1]:
# Libraries
# !pip install transformers
# !pip install dill
 

In [3]:
def remove_punctuations(texts):
  output = []
  for text in texts :
    x = re.sub(r'[^a-zA-Z ]','',text)
    output.append(x)
  return output

def handle_joined_words(texts):
  output = []
  for text in texts:
    x = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    output.append(x)
  return output


# Creating function transformer

remove_punctuations = FunctionTransformer(remove_punctuations,validate = False)
handle_joined_words = FunctionTransformer(handle_joined_words,validate = False)


In [2]:
# Creating CardiffNLPClassifier OBJECT
class CardiffNLPClassifier:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
        self.config = AutoConfig.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
        self.model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest").to(self.device)

    def transform(self, texts):
        results = []
        i = 0
        for text in texts:
            encoded_input = self.tokenizer(text, return_tensors='pt')
            encoded_input = {key: tensor.to(self.device) for key, tensor in encoded_input.items()}

            with torch.no_grad():
                output = self.model(**encoded_input)

            scores = output.logits.squeeze().detach().cpu().numpy()
            scores = softmax(scores)
            ranking = np.argsort(scores)[::-1]

            # Creating a dictionary with labels and probabilities
            d = {self.config.id2label[ranking[i]].upper(): np.round(float(scores[ranking[i]]), 4) for i in range(scores.shape[0])}

            # Removing 'neutral' class
            del d['NEUTRAL']

            # Extracting label with the highest probability
            key_with_highest_value = max(d, key=d.get)
            results.append(key_with_highest_value)
            if i % 100 == 0 :
              print(i)
            i = i + 1
        return results

sentiment_classifier = CardiffNLPClassifier()


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# class CardiffNLPClassifier:
#     def __init__(self):
#         self.tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
#         self.config = AutoConfig.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
#         self.model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

#     # def fit(self, texts, y=None):
#     #     return self

#     def transform(self, texts):
#         results = []
#         i = 0
#         for text in texts:
#             encoded_input = self.tokenizer(text, return_tensors='pt')
#             output = self.model(**encoded_input)
#             scores = output[0][0].detach().numpy()
#             scores = softmax(scores)
#             ranking = np.argsort(scores)[::-1]

#             # Creating a dictionary with labels and probabilities
#             d = {self.config.id2label[ranking[i]].upper(): np.round(float(scores[ranking[i]]), 4) for i in range(scores.shape[0])}

#             # Removing 'neutral' class
#             del d['NEUTRAL']

#             # Extracting label with the highest probability
#             key_with_highest_value = max(d, key=d.get)
#             results.append(key_with_highest_value)
#             if (i % 100) == 0:
#               print(i)
#             i = i+1
#         return results

# sentiment_classifier = CardiffNLPClassifier()

In [4]:
# Example
sentiment_analysis_pipeline = Pipeline([('remove_punctuations',remove_punctuations),('handle_joined_words',handle_joined_words),('sentiment_classifier',sentiment_classifier)])
X = ['lovely !!!','terrible','WOW I JUSTLOVED THIS','service is fine']
processed_text = sentiment_analysis_pipeline.transform(X)
print(processed_text)

0
['POSITIVE', 'NEGATIVE', 'POSITIVE', 'POSITIVE']


In [7]:
# saving in form of pickle file
with open('/content/drive/MyDrive/DATASETS/sentiment_analysis_pipeline.pkl', 'wb') as f:
    dill.dump(sentiment_analysis_pipeline, f)

In [5]:
df = pd.read_csv(r"C:\Users\YASH KHADTARE\FERGUSSON\SEM 3\Final year Project\DATA_YASH.csv")
df = df.sample(500)
df.drop('Unnamed: 0',axis=1,inplace=True)
df

,Brand_name,model_name,price,review,rating_by_reviewer
45251,SAMSUNG,"SAMSUNG Galaxy J6 (Blue, 32 GB)","₹11,500",nice,5
63218,SAMSUNG,SAMSUNG GT-E1215ZKAINS,"₹1,150",good,3
61966,SAMSUNG,SAMSUNG GT-E1215ZKAINS,"₹1,150",Good,5
47052,SAMSUNG,"SAMSUNG Galaxy F04 (Opal Green, 64 GB)","₹8,499",Overall phone is okThe only problem is the cha...,3
85951,VIVO,"vivo Y33T (Mirror Black, 128 GB)","₹23,000",I absolutely loved the phone ❣️ the overall pe...,5
...,...,...,...,...,...
77818,VIVO,"vivo V11 (Starry Night Black, 64 GB)","₹24,990",buy real me 2 with 8 GB ram if you want to use...,5
100151,VIVO,"vivo U10 (Electric Blue, 64 GB)","₹10,999",Awesome product 🤩🤩 Thanks to Vivo 😍,5
25869,SAMSUNG,"SAMSUNG Galaxy A21s (Black, 64 GB)","₹19,999",Good,5
16869,SAMSUNG,"SAMSUNG Galaxy A52 (Awesome Blue, 128 GB)","₹28,990","Bit Overpriced,Nothing special.Samsung is char...",3


In [6]:
# Predictions
df['Sentiment'] = sentiment_analysis_pipeline.transform(df['review'])

0


KeyboardInterrupt: 

In [10]:
df.head(5)

,Brand_name,model_name,price,review,rating_by_reviewer,Sentiment
0,SAMSUNG,SAMSUNG Galaxy Z Fold5,"₹1,84,999","Great phone, really good looking and specimen ...",5,POSITIVE
1,SAMSUNG,SAMSUNG Galaxy Z Fold5,"₹1,84,999",Hinge is good and phone is handyCG glass Victu...,5,POSITIVE
2,SAMSUNG,SAMSUNG Galaxy Z Fold5,"₹1,84,999","The phone is an actual testament of Ultra, Pro...",5,POSITIVE
3,SAMSUNG,SAMSUNG Galaxy Z Fold5,"₹1,84,999",Nice looking ND battery performance is good bu...,4,POSITIVE
4,SAMSUNG,SAMSUNG Galaxy Z Fold4 5G,"₹1,64,999",All good,5,POSITIVE


In [11]:
#Saving DATAFRAME
df.to_csv('/content/drive/MyDrive/DATASETS/FINAL_DATA_YASH.csv', index = False)

In [12]:
STOP = time.time()
print("Time required to run notebook : " , (STOP-START)/60, " mins")

Time required to run notebook :  25.321378147602083  mins
